In [1]:
import pandas as pd
import csv
import re
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
import gensim
from gensim import corpora, models
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import time
from pprint import pprint

stemmer = SnowballStemmer("english") # the stopwords 
def remain_english(line):
    rule =re.compile("[^a-z^A-Z]")
    line = rule.sub('',line)
    return line
def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v')) # remain problems

# Tokenize and lemmatize
def preprocess(text):
    result=[]
    for token in gensim.utils.simple_preprocess(str(text)) :
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
#             result.append(lemmatize_stemming(token))
            result.append(remain_english(lemmatize_stemming((token))).lower()) # take out not english and return into verb, and lower it 
    return result

c:\users\administrator\appdata\local\programs\python\python37\lib\site-packages\gensim\utils.py:1212: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
# read file into pandas
import pandas as pd
fileGP = r'C:\Users\Administrator\Desktop\reviewData\Reviews(GP)\LifeStyleReview.csv'
fileApple = r'C:\Users\Administrator\Desktop\reviewData\Reviews(Ap)\lifestylereviews.csv'
# file = r'C:\Users\Administrator\Desktop\reviewData\dist\lifestyle-dist.csv'
df1 = pd.read_csv(fileGP,error_bad_lines=False)#  Error tokenizing data. C error: Expected 13 fields in line 8791, saw 25
df2 = pd.read_csv(fileApple,encoding = "ISO-8859-1")

c:\users\administrator\appdata\local\programs\python\python37\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (4,5,19,20) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
gpNum = df1.count()['text']
ApNum = df2.count()['text'] # text(reviews) number from apple
sid = SentimentIntensityAnalyzer()
text_num = gpNum + ApNum
# text_num = 5500
documentNeg=[]
documentPos=[]
inconsistantNum1 = 0 # inconsistant reviews number in low rating reviews
inconsistantNum2 = 0
for i in range(gpNum):
    if(df1.score[i]<3): # low rating reviews
#         print(sid.polarity_scores(df1.text[i])['neg'])
        if(sid.polarity_scores(str(df1.text[i]))['pos'] > 0.5): # 如果是这条评论的sensitive分析分数显示这条评论其实是负面的,threhold = 0.5
            inconsistantNum1 += 1
        else:
            documentNeg.append(df1.text[i])             

    if(df1.score[i]>3): # high rating reviews
        if(sid.polarity_scores(str(df1.text[i]))['neg'] > 0.5): # if the neg score is high, not use this review,threhold = 0.5
            inconsistantNum2 += 1
        else:
            documentPos.append(df1.text[i]) 
            
for j in range(ApNum):
    if(df2.score[j]<3): # low rating reviews
        if(sid.polarity_scores(str(df2.text[j]))['pos'] > 0.5): # 如果是这条评论的sensitive分析分数显示这条评论其实是负面的,threhold = 0.5
            inconsistantNum1 += 1
        else:
            documentNeg.append(df2.text[j])             
    if(df2.score[j]>3): # low rating reviews
        if(sid.polarity_scores(str(df2.text[j]))['neg'] > 0.5): # if the neg score is high, not use this review,threhold = 0.5
            inconsistantNum2 += 1
        else:
            documentPos.append(df2.text[j])      

print('number of reviews text:',text_num)
print('number of neg:',len(documentNeg))
print('number of inconsistant neg reviews text(actually a pos reviews):',inconsistantNum1)
print('number of pos:',len(documentPos))
print('number of inconsistant pos reviews text(actually an neg reviews):',inconsistantNum2)

number of reviews text: 109212
number of neg: 33328
number of inconsistant neg reviews text(actually a pos reviews): 178
number of pos: 67267
number of inconsistant pos reviews text(actually an neg reviews): 143


In [4]:
# negtive reviews analyse
document = documentNeg
doc_clean = [preprocess(doc) for doc in document]

dictionary = corpora.Dictionary(doc_clean)

# 使用上面的词典，将转换文档列表（语料）变成 DT 矩阵
doc_term_matrix = [dictionary.doc2bow(doc) for doc in doc_clean]
print('made doc_term_matrix')

Lda = gensim.models.ldamodel.LdaModel
print('object num',len(doc_term_matrix))
start = time.time()
lda_model = gensim.models.LdaMulticore(doc_term_matrix, num_topics=10, id2word=dictionary, passes=100, workers=2)
# result = lda_model.print_topics(num_words=10)
end = time.time()
print ('Fitting Time:',end-start)
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))
# print(result)

made doc_term_matrix
object num 33328
Fitting Time: 2689.7548451423645
Topic: 0 
Words: 0.038*"game" + 0.031*"hate" + 0.027*"crap" + 0.025*"worst" + 0.025*"play" + 0.024*"stupid" + 0.020*"scan" + 0.018*"piec" + 0.017*"junk" + 0.015*"question"
Topic: 1 
Words: 0.021*"servic" + 0.020*"custom" + 0.018*"time" + 0.014*"point" + 0.012*"month" + 0.012*"card" + 0.011*"week" + 0.011*"money" + 0.011*"order" + 0.011*"support"
Topic: 2 
Words: 0.035*"time" + 0.033*"wast" + 0.028*"watch" + 0.027*"money" + 0.027*"free" + 0.025*"don" + 0.020*"download" + 0.019*"want" + 0.016*"like" + 0.016*"work"
Topic: 3 
Words: 0.058*"work" + 0.044*"updat" + 0.034*"open" + 0.032*"crash" + 0.029*"time" + 0.020*"camera" + 0.018*"load" + 0.016*"iphon" + 0.016*"screen" + 0.011*"great"
Topic: 4 
Words: 0.036*"like" + 0.025*"look" + 0.023*"peopl" + 0.020*"list" + 0.020*"search" + 0.014*"color" + 0.014*"pictur" + 0.013*"fake" + 0.012*"profil" + 0.012*"want"
Topic: 5 
Words: 0.037*"connect" + 0.037*"work" + 0.026*"bulb" + 

In [5]:
# positive
document = documentPos
doc_clean = [preprocess(doc) for doc in document]

dictionary = corpora.Dictionary(doc_clean)
# 使用上面的词典，将转换文档列表（语料）变成 DT 矩阵
doc_term_matrix = [dictionary.doc2bow(doc) for doc in doc_clean]
Lda = gensim.models.ldamodel.LdaModel
print('object num',len(doc_term_matrix))
start = time.time()
lda_model = gensim.models.LdaMulticore(doc_term_matrix, num_topics=10, id2word=dictionary, passes=100, workers=2)
# result = lda_model.print_topics(num_words=10)
end = time.time()
print ('Fitting Time:',end-start)
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))
# print(result)

object num 67267
Fitting Time: 4131.502308368683
Topic: 0 
Words: 0.071*"" + 0.017*"chore" + 0.016*"heat" + 0.015*"spin" + 0.013*"coffe" + 0.012*"auto" + 0.012*"prize" + 0.012*"relax" + 0.011*"okay" + 0.010*"case"
Topic: 1 
Words: 0.026*"time" + 0.023*"easi" + 0.021*"great" + 0.019*"money" + 0.018*"recommend" + 0.016*"servic" + 0.015*"point" + 0.014*"high" + 0.012*"card" + 0.010*"price"
Topic: 2 
Words: 0.051*"work" + 0.029*"great" + 0.018*"wallpap" + 0.016*"home" + 0.016*"light" + 0.015*"control" + 0.013*"easi" + 0.012*"connect" + 0.012*"watch" + 0.012*"devic"
Topic: 3 
Words: 0.025*"like" + 0.020*"color" + 0.018*"star" + 0.017*"updat" + 0.017*"time" + 0.015*"work" + 0.015*"review" + 0.012*"camera" + 0.010*"thing" + 0.009*"problem"
Topic: 4 
Words: 0.033*"help" + 0.019*"know" + 0.018*"time" + 0.018*"need" + 0.017*"great" + 0.016*"home" + 0.016*"locat" + 0.016*"search" + 0.014*"love" + 0.014*"famili"
Topic: 5 
Words: 0.181*"good" + 0.103*"great" + 0.072*"nice" + 0.033*"free" + 0.030*"l